# Imports

 Cover fundamentals of tensors using TensorFlow.
 
- Introduction to tensors
- Getting information from tensors
- Manipulating tensors
- Tensors and NumPy
- Using @tf.function (a way to speed up python)
- Using GPUs with TensorFlow (or TPUs)
- Exercises to try for yourself


## Introduction to Tensors

In [153]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import tensorflow_probability as tfp

In [154]:
print(tf.__version__)

2.15.0


In [155]:
# Creating Tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [156]:
# Check number of dimensions of tensor, ndim
scalar.ndim

0

In [157]:
# Create a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [158]:
vector.ndim

1

In [159]:
# create a matrix
matrix = tf.constant([[10,7],[7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [160]:
matrix.ndim

2

In [161]:
# create another matrix 
another_matrix = tf.constant([[[10.,7.],
                              [3.,2.]],
                              [[8.,9.],
                              [6.,2.]]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(2, 2, 2), dtype=float16, numpy=
array([[[10.,  7.],
        [ 3.,  2.]],

       [[ 8.,  9.],
        [ 6.,  2.]]], dtype=float16)>

In [162]:
another_matrix.ndim

3

In [163]:
tensor = tf.constant([[[1,2,3],
                       [4,5,6]],
                      [[7,8,9],
                       [10,11,12]],
                      [[13,14,15],
                       [16,17,18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [164]:
tensor.ndim

3

What we've created so far:
- scalar : single number
- vector: a number with direction
- matrix: 2-dimensional array of numbers
- tensor: n-dimensional array of numbers

### Creating tensors with tf.Variable

In [165]:
# create the same tensor but changeable
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [166]:
 # let's try and change one of the variables
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

- use constant if you dont want the variables to be changed
- use Variable when you want to change variables within the tensor
- Generally: When in doubt use constant and change later if you need

## Creating random Tensors

 Random tensors are tensors of some arbitrary size with random numbers

In [167]:
# Create random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42)
random_1 = random_1.normal(shape=(3,2))

In [168]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

In [169]:
random_2 == random_1

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True],
       [ True,  True]])>

### Shuffle the order of elements in a tensor

In [170]:
# valuable when you want to shuffle data so the inherent order doesn't effect learning
not_shuffled = tf.constant([[10,7],
                            [3,4],
                            [2,5]])
not_shuffled.ndim

2

In [171]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [172]:
# need a global and operation level seeds
tf.random.set_seed(42)
shuffled = tf.random.shuffle(not_shuffled, seed=42)
shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

### Other ways to create tensors

#### Creating tensors from NumPy arrays

In [173]:
tf.ones([10,7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [174]:
tf.zeros(shape=(3,4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

Tensors can be run on a GPU, that is the main difference to NumPy array

In [175]:
# turn numpy arrays into tensors
numpy_A = np.arange(1,25, dtype=np.int32)

# X = tf.constant(some_matrix) -- capital for matrix
# y = tf.constant(vector) -- lower case for vector

In [176]:
A = tf.constant(numpy_A, shape=(2,3,4)) # product of new shape has to have the same length of original matrix/vector
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

### Getting Info from Tensors

- Shape -- tensor.shape
- Rank --  tensor.ndim
- Axis and dimension -- tensor[0], tensor[:,1]...
- Size -- tf.size(tensor)

In [177]:
# Create a rank 4 tensor
rank_4_tensor = tf.zeros([2, 3, 4, 5])

In [178]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor) # size = 2*3*4*5

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [179]:
# Get various attributes of tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
tensor_size = tf.size(rank_4_tensor)
print("Total number of elements in our tensor:", tensor_size.numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our tensor: 120


### Indexing on Tensors

Just like python lists

In [180]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [181]:
# get the first dimension from each index, except for the final one
rank_4_tensor[:1,:1, :1, :0]

<tf.Tensor: shape=(1, 1, 1, 0), dtype=float32, numpy=array([], shape=(1, 1, 1, 0), dtype=float32)>

In [182]:
# Create a rank 2 tensor
rank_2_tensor = tf.constant([[10, 7],
                             [3, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [183]:
# Get last item of each row
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [184]:
# Adding an extra dimension
rank_3_tensor = rank_2_tensor[..., tf.newaxis] # ... every axis before the last one
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [185]:
# Alternative
tf.expand_dims(rank_2_tensor, axis=-1)

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

### Manipulating Tensors (tensor operations)

#### Basic Operations

In [186]:
# addition operator
tensor = tf.constant([[10, 7],
                      [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [187]:
# Multiplication
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [188]:
# Subtraction
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [189]:
# tensorflow function (library)
tf.multiply(tensor, 10)
# Use the function!

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [190]:
tf.add(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [191]:
tf.subtract(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

### Matrix Multiplication

In machine learning, this concept is very common

#### Dot Product

In [192]:
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [193]:
tf.matmul(tensor, tensor) # if yo use tensor * tensor it is element wise

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [194]:
tensor_2 = tf.constant([[1, 2, 5],
                        [7, 2, 1],
                        [3, 3, 3]])
tensor_2

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 2, 5],
       [7, 2, 1],
       [3, 3, 3]], dtype=int32)>

In [195]:
tensor_3 = tf.constant([[3, 5],
                        [6, 7],
                        [1, 8]])

In [196]:
tf.matmul(tensor_2, tensor_3)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [197]:
# python operator @
tensor_2 @ tensor_3

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [198]:
# inverse of matrix
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])

In [199]:
Y = tf.transpose(Y)

In [200]:
tf.matmul(X, Y)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [201]:
# Can also use reshape(X, shape=(2,3))

In [202]:
# perform dotproduct on x & y
tf.tensordot(X, Y, axes=1)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [203]:
tf.tensordot(Y, tf.reshape(X, shape=(3,2)), axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89, 116],
       [ 98, 128]], dtype=int32)>

In [204]:
print("Normal X:")
print(X, "\n")

print("X reshaed to (2, 3):")
print(tf.reshape(X, shape=(2,3)), "\n")

print("X transposed:")
print(tf.transpose(X))

Normal X:
tf.Tensor(
[[1 2]
 [3 4]
 [5 6]], shape=(3, 2), dtype=int32) 

X reshaed to (2, 3):
tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32) 

X transposed:
tf.Tensor(
[[1 3 5]
 [2 4 6]], shape=(2, 3), dtype=int32)


Generally, you should use transpose instead of reshape.

### Changing Tensor Datatypes

In [205]:
B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [206]:
C = tf.constant([7,10])
C.dtype

tf.int32

In [207]:
# change from float 32 to 16 --> reduced precision
D= tf.cast(B, dtype=tf.float16)
D.dtype

tf.float16

In [208]:
E = tf.cast(D, dtype=tf.float32)
E.dtype

tf.float32

### Aggregating tensors

Condensing them from multiple values down to a smaller amount of values.

In [209]:
# Getting the absolute values -- not the best
D = tf.constant([-7,-10])
D.numpy()

array([ -7, -10], dtype=int32)

In [210]:
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Let's go through the following forms of aggregation:
- minimum
- maximum
- mean
- sum

In [211]:
# Larger Tensor
tf.random.set_seed(42)
np.random.seed(42)
E = tf.constant(np.random.randint(0,100,size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([51, 92, 14, 71, 60, 20, 82, 86, 74, 74, 87, 99, 23,  2, 21, 52,  1,
       87, 29, 37,  1, 63, 59, 20, 32, 75, 57, 21, 88, 48, 90, 58, 41, 91,
       59, 79, 14, 61, 61, 46, 61, 50, 54, 63,  2, 50,  6, 20, 72, 38])>

In [212]:
tf.reduce_min(E).numpy()

1

In [213]:
tf.reduce_max(E).numpy()

99

In [214]:
tf.reduce_mean(E).numpy()

50

In [215]:
tf.reduce_sum(E).numpy()

2542

Challenge:
- variance
- standard deviation

In [216]:
E_float = tf.cast(E, dtype=tf.float16)

In [217]:
tf.math.reduce_variance(E_float)

<tf.Tensor: shape=(), dtype=float16, numpy=775.5>

In [218]:
tf.math.reduce_std(E_float)

<tf.Tensor: shape=(), dtype=float16, numpy=27.84>

In [219]:
tfp.stats.variance(E)

<tf.Tensor: shape=(), dtype=int64, numpy=776>

In [220]:
tfp.stats.stddev(E_float)

<tf.Tensor: shape=(), dtype=float16, numpy=27.84>

### Find Positional Max and Min

In [221]:
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [222]:
# Find the position of the maximum value
max_position = tf.argmax(F)

# Find the position of the minimum value
min_position = tf.argmin(F)

# Print the positions
print("Position of the maximum value:", max_position.numpy(), "with value:", F[max_position].numpy())
print("Position of the minimum value:", min_position.numpy(), "with value:", F[min_position].numpy())

Position of the maximum value: 42 with value: 0.9671384
Position of the minimum value: 16 with value: 0.009463668


### Squeezing a tensor (removing all single dimensions)

In [223]:
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape = (1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [224]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [225]:
G_squeezed = tf.squeeze(G)
G_squeezed.shape

TensorShape([50])

### One hot encoding tensors

In [226]:
some_list = [0, 1, 2, 3]
print(len(some_list))
# one hot encode
tf.one_hot(some_list, depth = 4)

4


<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [227]:
# specify custom values for one hot encoding
tf.one_hot(some_list, depth =4, on_value="Yo I love deep learning", off_value="I also like to dance")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'Yo I love deep learning', b'I also like to dance',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'Yo I love deep learning',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'Yo I love deep learning', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'I also like to dance', b'Yo I love deep learning']],
      dtype=object)>

### More tensor math

- squaring
- log
- square root

In [228]:
H = tf.range(1,10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [229]:
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [230]:
H_float = tf.cast(H, dtype=tf.float32)

In [231]:
tf.math.sqrt(H_float)

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [232]:
tf.math.log(H_float)

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### TensorFlow and NumPy compatibility

In [233]:
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [234]:
# conver tensor back into numpy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [235]:
# Conver tensor J to NumPy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [236]:
J = tf.constant([3.])

In [237]:
J.numpy()[0]

3.0

In [238]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### 🚀 Making sure our tensor operations run really fast on GPU

In [239]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [240]:
# for colab checking which GPU
# !nvidia-smi

In [241]:
# How do I get my tensorflow code to run on it?
# If you have a CUDA enabled GPU, TensorFlow will automatically use it whenever possible